<a href="https://colab.research.google.com/github/zeynep68/deep_learning/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Model 
from tensorflow.keras.optimizers import *
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import *
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras

Using TensorFlow backend.


In [0]:
class CIFAR10():
    def __init__(self):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = keras.datasets.cifar10.load_data()

        self.width = self.x_train.shape[1]
        self.height = self.x_train.shape[2]
        self.depth = self.x_train.shape[3]
        
        self.num_classes = np.max(self.y_train) + 1
        
        self.train_size = self.x_train.shape[0]
        self.test_size = self.x_test.shape[0]
        
        self.y_train = to_categorical(self.y_train, self.num_classes)
        self.y_test = to_categorical(self.y_test, self.num_classes)

        self.scaler = StandardScaler()    

    def get_train_set(self):
        return self.x_train, self.y_train
    
    def get_test_set(self):
        return self.x_test, self.y_test

    # ZCA preprocessing
    def normalize_data(self):
        self.scaler.fit(self.x_train.reshape(-1, 1024))
    
        self.x_train = self.scaler.transform(self.x_train.reshape(-1, 1024))
        self.x_test = self.scaler.transform(self.x_test.reshape(-1,1024))

        self.x_train = self.x_train.reshape((self.train_size, self.width, self.height, self.depth))
        self.x_test = self.x_test.reshape((self.test_size, self.width, self.height, self.depth))

    # data augmentation we do horizontal flips and take random crops from image 
    # padded by 4 pixels on each side, filling missing pixels with reflections of original image. 
    def data_augmentation(self, X, size=5000):
        rand_numbers = np.random.randint(self.train_size, size=size)

        x_flipped = tf.image.flip_left_right(X[rand_numbers])
        y_flipped = self.y_train[rand_numbers]

       #x_augmented = tf.image.random_crop(x_flipped, 4)

        self.x_train = np.concatenate((self.x_train, x_flipped))
        self.y_train = np.concatenate((self.y_train, y_flipped))

        return self.x_train, self.y_train
        

In [0]:
def conv_block(X, num_filters, strides):
    
    x1 = Conv2D(num_filters, kernel_size=(3,3), strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv2D(num_filters, kernel_size=(3,3), strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)

    x2 = Conv2D(num_filters, kernel_size=(3,3), strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x2 = Conv2D(num_filters, kernel_size=(3,3), strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x2)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)

    x = Add()([x1,x2])
    x = Activation('relu')(x)

    return x

In [0]:
def identity_block(X, num_filters):

    num_filters = X.shape[3]

    x = Conv2D(num_filters, kernel_size=(3,3), strides=(1,1), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(X)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Dropout(0.1)(x)

    x = Conv2D(num_filters, kernel_size=(3,3), strides=(1,1), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Add()([X, x])
    x = Activation('relu')(x)

    return x

In [0]:
def lr_schedule(epoch):
    lr = 1e-3

    if(epoch > 100):
        lr *= 1e-3
    elif epoch > 75:
        lr *= 1e-2
    elif epoch >= 40:
        lr *= 1e-1

    return lr

In [0]:
def create_model(x):
    input_img = tf.keras.Input(shape=x.shape[1:])

    x = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Dropout(0.1)(x)

    x = conv_block(x, 64, (1,1))
    x = identity_block(x, 64)

    x = Dropout(0.15)(x)

    x = conv_block(x, 128, (1,1))

    x = identity_block(x, 128)

    x = Dropout(0.2)(x)

    x = conv_block(x, 256, (2,2))

    x = identity_block(x, 256)

    x = Dropout(0.25)(x)

    x = AveragePooling2D(pool_size=(8,8), padding='valid')(x)

    x = Flatten()(x)
    y_pred = Dense(10, activation='softmax')(x)

    model = Model(inputs=[input_img], outputs=[y_pred])


    model.compile(
                  optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                 )
    
    return model

In [7]:
cifar = CIFAR10()
cifar.normalize_data()

x_train, y_train = cifar.get_train_set()
x_test, y_test = cifar.get_test_set()

#x_train, y_train = cifar.data_augmentation(x_train)

lr_scheduler = LearningRateScheduler(lr_schedule)

model = create_model(x_train)

EPOCHS = 120
BATCH_SIZE = 128

training = model.fit(
                     x_train,
                     y_train,
                     batch_size=BATCH_SIZE, 
                     epochs=EPOCHS,
                     validation_data=(x_test, y_test),
                     callbacks=[lr_scheduler]
                    )

Epoch 1/120
391/391 [==============================] - 58s 148ms/step - loss: 1.8150 - accuracy: 0.5344 - val_loss: 3.8111 - val_accuracy: 0.2427 - lr: 0.0010
Epoch 2/120
391/391 [==============================] - 57s 145ms/step - loss: 1.3055 - accuracy: 0.6929 - val_loss: 1.4590 - val_accuracy: 0.6464 - lr: 0.0010
Epoch 3/120
391/391 [==============================] - 57s 145ms/step - loss: 1.0809 - accuracy: 0.7539 - val_loss: 1.6856 - val_accuracy: 0.5832 - lr: 0.0010
Epoch 4/120
391/391 [==============================] - 57s 145ms/step - loss: 0.9289 - accuracy: 0.7941 - val_loss: 1.2646 - val_accuracy: 0.6740 - lr: 0.0010
Epoch 5/120
391/391 [==============================] - 57s 146ms/step - loss: 0.8397 - accuracy: 0.8150 - val_loss: 1.8676 - val_accuracy: 0.5682 - lr: 0.0010
Epoch 6/120
391/391 [==============================] - 57s 145ms/step - loss: 0.7809 - accuracy: 0.8320 - val_loss: 1.7748 - val_accuracy: 0.6479 - lr: 0.0010
Epoch 7/120
391/391 [=========================